In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
# get data from ticker in set100

ticker_list = pd.read_csv('set100.csv')
ticker_list = ticker_list['Symbol'].tolist()

df = pd.DataFrame()
df_info = pd.DataFrame()
for ticker in ticker_list:
    this_ticker = yf.Ticker(ticker + ".BK")
    ticker_info = this_ticker.info
    print(ticker)
    df = df.append(this_ticker.history(period="5y", interval="1d",auto_adjust=False , rounding=True).assign(ticker=ticker))

df.to_csv('set100_price.csv')

ACE
ADVANC
AEONTS
AMATA
AOT
AP
AWC
BAM
BANPU
BBL
BCH
BCP
BCPG
BDMS
BEC
BEM
BGRIM
BH
BLA
BTS
CBG
CENTEL
CHG
CK
CKP
COM7
CPALL
CPF
CPN
CRC
DOHOME
DTAC
EA
EGCO
EPG
ESSO
FORTH
GLOBAL
GPSC
GULF
GUNKUL
HANA
HMPRO
INTUCH
IRPC
IVL
JMART
JMT
KBANK
KCE
KEX
KKP
KTB
KTC
LH
MAJOR
MEGA
MINT
MTC
ONEE
OR
ORI
OSP
PLANB
PSL
PTG
PTT
PTTEP
PTTGC
QH
RATCH
RBF
RCL
SAWAD
SCB
SCC
SCGP
SINGER
SPALI
SPRC
STA
STARK
STEC
STGT
SUPER
SYNEX
TASCO
TCAP
THANI
TIDLOR
TIPH
TISCO
TOP
TQM
TRUE
TTA
TTB
TU
VGI
WHA


In [2]:
df = pd.read_csv('set100_price.csv')
df["Close"] = df["Close"].astype(float)
df[["Date", "ticker", "Close"]].set_index("Date")

,ticker,Close
Date,,
2019-11-13,ACE,4.64
2019-11-14,ACE,4.28
2019-11-15,ACE,4.20
2019-11-18,ACE,4.54
2019-11-19,ACE,4.64
...,...,...
2022-10-26,WHA,3.74
2022-10-27,WHA,3.86
2022-10-28,WHA,3.92


In [3]:
# pivot on ticker
df_pivot = df.pivot(columns="ticker", values="Close", index="Date").copy()
df_pivot.to_csv('set100_price_pivot.csv')
df_pivot.shape

(1214, 100)

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import math

PATH = "C:\Program Files (x86)\chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(PATH)

set100 = pd.read_csv("set100.csv")
set100_ticker = set100["Symbol"].tolist()

df_info = pd.DataFrame()

for ticker in set100_ticker:

    driver.get("https://www.set.or.th/th/market/product/stock/quote/" + str(ticker) + "/financial-statement/company-highlights")
    try:
        element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, "//*[contains(text(),'P/E (เท่า)')]"))
                    )
        print(ticker)
            
        def get_PE(i):
            # scrape P/E data from set.or.th (i is the column number)
            try:
                result = driver.find_element_by_xpath("//*[contains(text(),'P/E (เท่า)')]/ancestor::tr[@indexselected='3']/td[@aria-colindex='" + str(i) + "']/div/div").text
                return float(result) if result != "-" else np.nan
            except:
                return np.nan
        
        def get_PE_2022():
            # get trailing pe from yfinance (Nov 2022)
            try:
                return round(yf.Ticker(ticker + ".BK").info["trailingPE"], 2)
            except:
                return np.nan

        df_info = df_info.append({"Symbol": ticker, "PE2018": get_PE(2), "PE2019": get_PE(3), "PE2020": get_PE(4), "PE2021": get_PE(5), "PE2022": get_PE_2022()}, ignore_index=True)
    except:
        print("error")

# get average PE in new column
df_info["AvgPE"] = df_info[["PE2018", "PE2019", "PE2020", "PE2021", "PE2022"]].mean(axis=1).round(2)

df_info.to_csv("set100_info.csv", index=False)

In [3]:
df_pivot = pd.read_csv("set100_price_pivot.csv")
df_pivot["Date"] = pd.to_datetime(df_pivot["Date"])
df_pivot = df_pivot.set_index("Date")
# df_pivot = df_pivot.dropna(axis=1, how="any")

In [4]:
Rf = 2.795 # risk free rate
n = 365

# daily risk free rate
Rf_daily = ((1 + Rf/100)**(1/n) - 1) * 100

In [5]:
df_info = pd.read_csv("set100_info.csv")
avgPE = df_info["AvgPE"].tolist()
ticker = df_info["Symbol"].tolist()

In [6]:
# percentile 75th of avgPE
avgPE_list = df_info["AvgPE"].tolist()
sorted_avgPE = sorted(avgPE_list)
avgPE_75th = sorted_avgPE[int(len(sorted_avgPE) * 0.75)]
print(avgPE_75th)

33.44


In [7]:
# select only stocks with avgPE < 75th percentile and current ratio between 1.25 and 2
selected_ticker = []
for i in range(len(ticker)):
    if avgPE[i] <= avgPE_75th:
        selected_ticker.append(ticker[i])
len(selected_ticker)

76

In [8]:
df_selected = df_pivot[selected_ticker]
df_selected = df_selected.dropna(axis=1, how="any")
df_selected.head()

,ADVANC,AEONTS,AMATA,AP,BANPU,BBL,BCH,BCP,BCPG,BEC,...,SYNEX,TASCO,TCAP,THANI,TISCO,TOP,TTA,TTB,TU,WHA
Date,,,,,,,,,,,,,,,,,,,,,
2017-11-01,193.0,107.5,21.4,9.20,17.4,194.5,16.5,40.50,23.7,16.9,...,14.36,22.2,54.00,4.4121,88.00,98.75,9.75,2.3960,18.0,3.76
2017-11-02,187.5,107.0,21.1,9.20,17.3,192.0,16.3,39.50,23.5,16.7,...,14.27,22.1,52.50,4.3644,88.25,97.75,9.45,2.3777,17.9,3.74
2017-11-03,184.5,106.5,21.0,9.00,17.1,192.0,16.5,40.25,23.5,16.8,...,14.18,21.7,52.00,4.4121,86.00,98.00,9.65,2.3594,17.9,3.78
2017-11-06,186.5,107.0,21.9,9.20,17.5,192.0,16.6,40.00,23.7,17.2,...,14.73,21.9,53.75,4.3644,87.75,98.50,9.65,2.3777,18.9,3.92
2017-11-07,186.5,107.0,21.7,9.25,17.5,190.0,16.9,40.00,23.4,17.1,...,14.64,22.0,54.00,4.4598,87.00,99.00,9.80,2.3594,18.6,3.88


In [9]:
# get daily returns rate
df_returns = df_selected.pct_change().dropna() * 100
mean_returns = np.mean(df_returns, axis=0)
print(mean_returns)
cov_returns = np.cov(df_returns, rowvar=False)
print(cov_returns)

ADVANC    0.009434
AEONTS    0.070728
AMATA     0.022261
AP        0.024993
BANPU     0.011406
            ...   
TOP      -0.017922
TTA       0.027090
TTB      -0.022052
TU        0.018294
WHA       0.024336
Length: 64, dtype: float64
[[1.84515845 0.73274486 0.96584438 ... 0.91315622 0.42400369 0.81084694]
 [0.73274486 7.28811656 2.02603844 ... 1.87180633 0.55441571 2.10504657]
 [0.96584438 2.02603844 5.95946122 ... 2.13213342 1.09462539 3.66684239]
 ...
 [0.91315622 1.87180633 2.13213342 ... 4.74091514 0.80830735 1.90926938]
 [0.42400369 0.55441571 1.09462539 ... 0.80830735 3.46687819 0.91108633]
 [0.81084694 2.10504657 3.66684239 ... 1.90926938 0.91108633 4.48098262]]


In [10]:
# original code by Vijayalakshmi Pai G. A.
# https://github.com/PaiViji/PythonFinance-PortfolioOptimization/tree/master/Lesson6_SharpeRatioOptimization

from scipy import optimize 

def MaximizeSharpeRatioOptmzn(MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
    
    # define maximization of Sharpe Ratio using principle of duality
    def  f(x, MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
        funcDenomr = np.sqrt(np.matmul(np.matmul(x, CovarReturns), x.T) )
        funcNumer = np.matmul(np.array(MeanReturns),x.T)-RiskFreeRate
        func = -(funcNumer / funcDenomr)
        return func

    #define equality constraint representing fully invested portfolio
    def constraintEq(x):
        A=np.ones(x.shape)
        b=1
        constraintVal = np.matmul(A,x.T)-b 
        return constraintVal
    
    #define bounds and other parameters
    xinit=np.repeat(0.33, PortfolioSize)
    cons = ({'type': 'eq', 'fun':constraintEq})
    lb = 0
    ub = 0.1
    bnds = tuple([(lb,ub) for x in xinit])
    
    #invoke minimize solver
    opt = optimize.minimize (f, x0 = xinit, args = (MeanReturns, CovarReturns,\
                             RiskFreeRate, PortfolioSize), method = 'SLSQP',  \
                             bounds = bnds, constraints = cons, tol = 10**-3)
    
    return opt

In [11]:
portfolioSize = len(df_selected.columns)
portfolioSize

64

In [12]:
xOptimal =[]
minRiskPoint = []
expPortfolioReturnPoint =[]
maxSharpeRatio = 0

result = MaximizeSharpeRatioOptmzn(mean_returns, cov_returns, Rf_daily, portfolioSize)
xOptimal.append(result.x)

xOptimalArray = np.array(xOptimal)
Risk = np.matmul((np.matmul(xOptimalArray,cov_returns)), np.transpose(xOptimalArray))
expReturn = np.matmul(np.array(mean_returns),xOptimalArray.T)
annRisk =   np.sqrt(Risk*251) 
annRet = 251*np.array(expReturn) 
maxSharpeRatio = (annRet-Rf)/annRisk 

In [13]:
#set precision for printing results
np.set_printoptions(precision=3, suppress = True)
xOptimalArray = xOptimalArray*100

#display results
print('Maximal Sharpe Ratio: ', maxSharpeRatio, '\nAnnualized Risk (%):  ', \
      annRisk, '\nAnnualized Expected Portfolio Return(%):  ', annRet)
print('\nOptimal weights (%):\n', xOptimalArray )

Maximal Sharpe Ratio:  [[1.072]] 
Annualized Risk (%):   [[20.651]] 
Annualized Expected Portfolio Return(%):   [24.925]

Optimal weights (%):
 [[ 0.471  3.63   0.     0.     0.     0.     1.308  0.     0.     0.
   0.     0.     0.     7.685  0.    10.     0.     0.     0.     0.
   0.     7.538  0.     4.979  0.     0.     0.     0.     0.     0.
  10.     0.     0.     0.     0.     0.     0.     2.403  0.     0.
  10.     0.     0.     0.    10.     0.     0.    10.     0.     0.
   9.186 10.     0.     0.     0.     0.     0.     0.     0.729  0.
   0.     0.     2.07   0.   ]]


In [43]:

# append pivot columns to xOptimalArray
res_df = pd.DataFrame(np.append(xOptimalArray.reshape(-1,1).round(2) , df_selected.columns.values.reshape(-1,1), axis=1), columns=["Weight", "Symbol"])


In [44]:
res_df.to_csv("set100_optimal_weights.csv", index=False)